# LangGraph Hierarchical Teams
## Tech Startup Due Diligence System

In the [supervisor notebook](langgraph_multi_agent_supervisor.ipynb), we built a single supervisor managing multiple agents. But what happens when your system grows to 8, 10, or 15 agents? A single supervisor becomes a bottleneck — it has too many agents to manage and too much context to track.

The solution: **hierarchical teams** — teams of teams, where each sub-team has its own supervisor, and a top-level orchestrator coordinates between sub-teams.

### Architecture Overview

```
                    ┌─────────────┐
                    │ Orchestrator│
                    └──────┬──────┘
                     ┌─────┴──────┐
              ┌──────┴──────┐  ┌──┴────────────┐
              │Market Research│  │Financial Analysis│
              │   Sub-Team   │  │    Sub-Team      │
              └──────┬──────┘  └──┬────────────┘
            ┌────────┴───┐    ┌───┴──────────┐
         Market    Competitor  Revenue     Risk
         Analyst   Analyst    Analyst    Analyst
```

### What We're Building

A due diligence system for evaluating tech startup investments:
- **Market Research Team**: Market analyst + Competitor analyst, managed by a team supervisor
- **Financial Analysis Team**: Revenue analyst + Risk analyst, managed by a team supervisor  
- **Orchestrator**: Coordinates both teams and synthesizes a final investment report

### Key Concepts
- **Subgraph composition** — compile a graph, then use it as a node in a parent graph
- **State mapping** — translate between parent and child state schemas
- **Layered supervision** — each level manages its own scope

In [ ]:
%pip install -q langgraph langchain langchain-openai langchain-community

In [ ]:
import os
import getpass
import operator
from typing import Annotated, TypedDict, Literal, Any

from pydantic import BaseModel, Field

from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("Setup complete!")

## 1. State Design: Parent vs Child

A key challenge in hierarchical systems is **state boundaries**. Each sub-team has its own internal state, but the parent orchestrator needs to see a summary of the team's output — not every internal message.

We define two state types:
- **`SubTeamState`** — Used within each sub-team. Contains messages and routing info.
- **`OrchestratorState`** — Used at the top level. Contains summary outputs from each team.

In [ ]:
# Sub-team internal state
class SubTeamState(TypedDict):
    messages: Annotated[list, add_messages]
    next: str
    team_output: str  # Final summary from this team

# Top-level orchestrator state
class OrchestratorState(TypedDict):
    messages: Annotated[list, add_messages]
    next: str
    market_research_output: str
    financial_analysis_output: str
    final_report: str

print("State types defined:")
print(f"  SubTeamState fields: {list(SubTeamState.__annotations__.keys())}")
print(f"  OrchestratorState fields: {list(OrchestratorState.__annotations__.keys())}")

## 2. Building the Market Research Sub-Team

Our first sub-team specializes in market research. It has:
- **Market Analyst** — Analyzes market size, growth trends, and opportunities
- **Competitor Analyst** — Maps the competitive landscape
- **Team Supervisor** — Routes between analysts and compiles the team output

### Sub-Team Tools

Each analyst has domain-specific mock tools that simulate access to market data.

In [ ]:
@tool
def analyze_market_size(industry: str) -> str:
    """Analyze the total addressable market (TAM) for an industry."""
    markets = {
        "ai healthcare": "TAM: $45.2B (2024), projected $187.9B by 2030. CAGR: 26.7%. Key segments: diagnostics (35%), drug discovery (28%), clinical trials (20%), admin automation (17%).",
        "ai": "TAM: $196.6B (2024), projected $1.81T by 2030. CAGR: 44.7%. Broad market with rapid growth across all segments.",
        "fintech": "TAM: $305.7B (2024), projected $556.6B by 2030. CAGR: 10.5%. Mature but still growing.",
        "saas": "TAM: $197B (2024), projected $232B by 2025. CAGR: 18%. Increasingly competitive market.",
    }
    for key, value in markets.items():
        if key in industry.lower():
            return value
    return f"Market data not available for: {industry}. Consider a broader industry category."

@tool
def get_industry_trends(industry: str) -> str:
    """Get current trends and growth drivers for an industry."""
    trends = {
        "ai healthcare": "Key trends: 1) FDA accelerating AI/ML device approvals (171 cleared in 2023), 2) Hospital AI adoption at 40% and rising, 3) Shift from rule-based to foundation models, 4) Interoperability mandates driving data availability, 5) Payer reimbursement for AI-assisted diagnostics expanding.",
        "ai": "Key trends: 1) Foundation model costs dropping rapidly, 2) Enterprise adoption accelerating, 3) Regulatory frameworks emerging globally, 4) Open-source vs proprietary tensions, 5) AI agent paradigm emerging.",
        "fintech": "Key trends: 1) Embedded finance growth, 2) Open banking regulations, 3) BNPL market maturation, 4) Crypto/DeFi institutional adoption, 5) AI-driven risk assessment.",
    }
    for key, value in trends.items():
        if key in industry.lower():
            return value
    return f"No trend data available for: {industry}"

@tool  
def analyze_competitors(company_name: str) -> str:
    """Analyze the competitive landscape for a company."""
    return f"""Competitive analysis for {company_name}:
Direct competitors: 3-5 established players with $50M+ revenue
Indirect competitors: Large tech companies (Google Health, Microsoft Health) entering space
Key differentiators to evaluate: proprietary data moats, regulatory approvals, clinical validation
Barrier to entry: HIGH (regulatory requirements, clinical validation needed, data access)
Market concentration: Fragmented — top 5 players hold ~30% market share"""

@tool
def assess_competitive_advantage(advantage_type: str) -> str:
    """Assess a specific type of competitive advantage."""
    advantages = {
        "data moat": "Data moat assessment: STRONG competitive advantage in healthcare AI. Proprietary clinical datasets are extremely difficult to replicate. Companies with 1M+ patient records have 2-3 year head start.",
        "technology": "Technology assessment: MODERATE advantage. AI model architectures are increasingly commoditized. Advantage comes from domain-specific fine-tuning and clinical validation, not raw model capability.",
        "regulatory": "Regulatory assessment: STRONG advantage. FDA 510(k) or De Novo clearance takes 6-18 months. Each cleared indication is a durable moat. Competitors must independently validate.",
        "network": "Network effects assessment: MODERATE. More hospital deployments → more data → better models → easier sales. But network effects are slower than in consumer tech.",
    }
    return advantages.get(advantage_type.lower(), f"No assessment available for advantage type: {advantage_type}")

market_tools = [analyze_market_size, get_industry_trends]
competitor_tools = [analyze_competitors, assess_competitive_advantage]

print("Market Research tools defined:")
print("  Market Analyst: analyze_market_size, get_industry_trends")
print("  Competitor Analyst: analyze_competitors, assess_competitive_advantage")

In [ ]:
market_analyst_llm = llm.bind_tools(market_tools)
competitor_analyst_llm = llm.bind_tools(competitor_tools)

def market_analyst(state: SubTeamState) -> dict:
    """Analyzes market size, growth, and trends."""
    response = market_analyst_llm.invoke([
        SystemMessage(content="""You are a senior market analyst specializing in technology markets. 
Your job is to assess market size (TAM/SAM/SOM), growth rates, and key trends.
Use your tools to gather data, then provide a concise analysis.
Focus on quantitative data and specific numbers."""),
        *state["messages"],
    ])
    
    if response.tool_calls:
        from langgraph.prebuilt import ToolNode
        tool_results = ToolNode(market_tools).invoke({"messages": [response]})
        follow_up = llm.invoke([
            SystemMessage(content="You are a market analyst. Synthesize the market data into a clear analysis with key metrics."),
            *state["messages"], response, *tool_results["messages"],
        ])
        return {"messages": [AIMessage(content=f"[Market Analyst] {follow_up.content}")]}
    
    return {"messages": [AIMessage(content=f"[Market Analyst] {response.content}")]}

def competitor_analyst(state: SubTeamState) -> dict:
    """Maps competitive landscape and evaluates moats."""
    response = competitor_analyst_llm.invoke([
        SystemMessage(content="""You are a competitive intelligence analyst. 
Your job is to map the competitive landscape, identify key players, and assess competitive advantages.
Use your tools to analyze competitors and evaluate different types of competitive moats.
Focus on defensibility and differentiation."""),
        *state["messages"],
    ])
    
    if response.tool_calls:
        from langgraph.prebuilt import ToolNode
        tool_results = ToolNode(competitor_tools).invoke({"messages": [response]})
        follow_up = llm.invoke([
            SystemMessage(content="You are a competitive analyst. Synthesize the competitive landscape into a clear assessment."),
            *state["messages"], response, *tool_results["messages"],
        ])
        return {"messages": [AIMessage(content=f"[Competitor Analyst] {follow_up.content}")]}
    
    return {"messages": [AIMessage(content=f"[Competitor Analyst] {response.content}")]}

print("Market Research agent nodes defined: market_analyst, competitor_analyst")

In [ ]:
class TeamRouteDecision(BaseModel):
    """Routing decision within a sub-team."""
    next: str = Field(description="Next agent or FINISH")
    reasoning: str = Field(description="Why this routing")

market_team_structured_llm = llm.with_structured_output(TeamRouteDecision)

def market_team_supervisor(state: SubTeamState) -> dict:
    """Supervisor for the market research sub-team."""
    decision = market_team_structured_llm.invoke([
        SystemMessage(content="""You supervise a market research team with two analysts:
- market_analyst: Analyzes market size, TAM, growth rates, industry trends
- competitor_analyst: Maps competitive landscape, evaluates moats and differentiation

Review the conversation. If market analysis is done, route to competitor analysis (or vice versa).
If both have reported, route to FINISH.
Always ensure both analysts contribute before finishing."""),
        *state["messages"],
    ])
    
    next_node = decision.next.lower().strip()
    if "finish" in next_node:
        # Compile team output before finishing
        summary = llm.invoke([
            SystemMessage(content="Compile a concise market research summary combining all analyst findings. Include key metrics and competitive assessment. 2-3 paragraphs max."),
            *state["messages"],
        ])
        return {
            "next": "FINISH",
            "team_output": summary.content,
            "messages": [AIMessage(content=f"[Market Research Supervisor] Team complete. Summary compiled.")],
        }
    
    # Map to valid agent names
    if "market" in next_node:
        return {"next": "market_analyst", "messages": [AIMessage(content="[Market Research Supervisor] Routing to market analyst.")]}
    elif "competitor" in next_node:
        return {"next": "competitor_analyst", "messages": [AIMessage(content="[Market Research Supervisor] Routing to competitor analyst.")]}
    else:
        return {"next": "market_analyst", "messages": [AIMessage(content="[Market Research Supervisor] Defaulting to market analyst.")]}

def route_market_team(state: SubTeamState) -> str:
    next_node = state.get("next", "FINISH")
    if next_node == "FINISH":
        return END
    return next_node

# Build Market Research subgraph
market_graph = StateGraph(SubTeamState)
market_graph.add_node("supervisor", market_team_supervisor)
market_graph.add_node("market_analyst", market_analyst)
market_graph.add_node("competitor_analyst", competitor_analyst)

market_graph.add_edge(START, "supervisor")
market_graph.add_conditional_edges("supervisor", route_market_team, {
    "market_analyst": "market_analyst",
    "competitor_analyst": "competitor_analyst",
    END: END,
})
market_graph.add_edge("market_analyst", "supervisor")
market_graph.add_edge("competitor_analyst", "supervisor")

market_research_app = market_graph.compile()
print("Market Research sub-team compiled!")

### Testing the Market Research Sub-Team Standalone

Before integrating into the parent graph, let's verify the sub-team works on its own.

In [ ]:
market_result = market_research_app.invoke({
    "messages": [HumanMessage(content="Evaluate the market opportunity for Acme AI Corp, which builds AI-powered diagnostic tools for hospitals. They operate in the AI healthcare space.")],
    "next": "",
    "team_output": "",
})

print("=" * 80)
print("MARKET RESEARCH SUB-TEAM (Standalone Test)")
print("=" * 80)
for msg in market_result["messages"]:
    if isinstance(msg, AIMessage):
        print(f"\n  {msg.content[:300]}")

print(f"\n{'─' * 60}")
print("TEAM OUTPUT:")
print(market_result.get("team_output", "No output")[:500])

## 3. Building the Financial Analysis Sub-Team

The second sub-team handles financial due diligence with:
- **Revenue Analyst** — Evaluates revenue model, unit economics, and growth metrics
- **Risk Analyst** — Assesses financial, market, and operational risks
- **Team Supervisor** — Coordinates analysis and compiles findings

In [ ]:
@tool
def analyze_revenue_model(model_type: str) -> str:
    """Analyze a specific revenue model type for a startup."""
    models = {
        "saas": "SaaS model assessment: Recurring revenue (strong). Key metrics: ARR growth >100% Y/Y is exceptional, >40% is good. Net revenue retention >120% indicates strong expansion. CAC payback <18 months is healthy. Gross margins 70-80% typical for healthcare SaaS.",
        "usage": "Usage-based model assessment: Variable revenue (moderate risk). Pros: lower barrier to adoption, natural expansion. Cons: less predictable, seasonal variation. Key metric: Net dollar retention and usage growth rate.",
        "enterprise": "Enterprise sales assessment: High ACV ($100K+), longer sales cycles (6-12 months). Key metrics: pipeline coverage >3x, win rate >25%, deal expansion rate. Requires strong sales org investment.",
        "hybrid": "Hybrid model (platform + usage): Combines base SaaS fee with usage charges. Best of both worlds if executed well. Requires careful pricing to avoid customer confusion.",
    }
    return models.get(model_type.lower(), f"No analysis available for revenue model: {model_type}")

@tool
def evaluate_unit_economics(metric: str) -> str:
    """Evaluate specific unit economics for a health-tech startup."""
    metrics = {
        "ltv": "Estimated LTV for enterprise health-tech: $250K-$500K per hospital contract. Based on 3-5 year retention, $80K-$120K ACV, with 10-20% annual expansion.",
        "cac": "Estimated CAC for enterprise health-tech: $50K-$100K per customer. Includes clinical validation costs, regulatory compliance, long sales cycles. LTV:CAC ratio of 3:1-5:1 is target.",
        "margins": "Gross margin benchmarks: 70-85% for pure software, 50-65% if hardware/devices included. Healthcare AI typically 65-75% due to clinical support costs.",
        "payback": "CAC payback period: 12-24 months typical for enterprise health-tech. Under 18 months is strong. Over 24 months raises concerns about capital efficiency.",
    }
    return metrics.get(metric.lower(), f"No unit economics data for: {metric}")

@tool
def assess_financial_risk(risk_type: str) -> str:
    """Assess a specific type of financial risk."""
    risks = {
        "market": "Market risk: MODERATE. AI healthcare is growing but regulatory changes could slow adoption. Reimbursement uncertainty for AI-assisted diagnoses. Potential for commoditization of base AI models.",
        "execution": "Execution risk: HIGH. Requires simultaneous excellence in AI/ML, clinical validation, regulatory compliance, and enterprise sales. Very few teams can execute across all four domains.",
        "regulatory": "Regulatory risk: HIGH. FDA approval timelines uncertain. Potential for stricter AI regulations in healthcare. EU AI Act classification could require significant compliance investment.",
        "competitive": "Competitive risk: MODERATE. Large tech incumbents (Google, Microsoft) could enter. But clinical validation and hospital relationships create meaningful barriers. Acqui-hire risk if team is strong but business is weak.",
        "financial": "Financial risk: MODERATE. High burn rate typical in healthcare AI ($2-5M/month). Long sales cycles require 18-24 months runway. Series B should provide 24+ months runway at current burn.",
    }
    return risks.get(risk_type.lower(), f"No risk assessment for: {risk_type}")

@tool
def benchmark_valuation(stage: str) -> str:
    """Get valuation benchmarks for a startup at a given stage."""
    benchmarks = {
        "series_b": "Series B AI healthcare benchmarks (2024): Median pre-money $150M-$300M. Revenue multiple: 20-40x ARR. Key milestones: $5M+ ARR, FDA clearance for at least one product, 20+ hospital customers.",
        "series_a": "Series A benchmarks: Pre-money $30M-$80M. Revenue multiple: 50-100x ARR (or pre-revenue based on milestones). Key: working product, initial clinical validation.",
        "series_c": "Series C benchmarks: Pre-money $400M-$1B. Revenue multiple: 15-25x ARR. Key: $20M+ ARR, clear path to profitability, market leadership position.",
    }
    return benchmarks.get(stage.lower().replace(" ", "_"), f"No benchmarks for stage: {stage}")

revenue_tools = [analyze_revenue_model, evaluate_unit_economics]
risk_tools = [assess_financial_risk, benchmark_valuation]

print("Financial Analysis tools defined:")
print("  Revenue Analyst: analyze_revenue_model, evaluate_unit_economics")
print("  Risk Analyst: assess_financial_risk, benchmark_valuation")

In [ ]:
revenue_analyst_llm = llm.bind_tools(revenue_tools)
risk_analyst_llm = llm.bind_tools(risk_tools)

def revenue_analyst(state: SubTeamState) -> dict:
    """Evaluates revenue model and unit economics."""
    response = revenue_analyst_llm.invoke([
        SystemMessage(content="""You are a revenue analyst specializing in tech startup due diligence.
Evaluate the company's revenue model, unit economics (LTV, CAC, margins, payback period), and growth trajectory.
Use your tools to get benchmark data. Provide specific numbers and comparisons."""),
        *state["messages"],
    ])
    
    if response.tool_calls:
        from langgraph.prebuilt import ToolNode
        tool_results = ToolNode(revenue_tools).invoke({"messages": [response]})
        follow_up = llm.invoke([
            SystemMessage(content="Synthesize the revenue analysis with specific metrics and a clear assessment."),
            *state["messages"], response, *tool_results["messages"],
        ])
        return {"messages": [AIMessage(content=f"[Revenue Analyst] {follow_up.content}")]}
    
    return {"messages": [AIMessage(content=f"[Revenue Analyst] {response.content}")]}

def risk_analyst(state: SubTeamState) -> dict:
    """Assesses financial, market, and operational risks."""
    response = risk_analyst_llm.invoke([
        SystemMessage(content="""You are a risk analyst specializing in tech startup due diligence.
Assess financial, market, regulatory, execution, and competitive risks.
Use your tools to get risk assessments and valuation benchmarks.
Rate each risk category (LOW/MODERATE/HIGH) with specific reasoning."""),
        *state["messages"],
    ])
    
    if response.tool_calls:
        from langgraph.prebuilt import ToolNode
        tool_results = ToolNode(risk_tools).invoke({"messages": [response]})
        follow_up = llm.invoke([
            SystemMessage(content="Synthesize the risk assessment with clear ratings and specific concerns for each category."),
            *state["messages"], response, *tool_results["messages"],
        ])
        return {"messages": [AIMessage(content=f"[Risk Analyst] {follow_up.content}")]}
    
    return {"messages": [AIMessage(content=f"[Risk Analyst] {response.content}")]}

print("Financial Analysis agent nodes defined: revenue_analyst, risk_analyst")

In [ ]:
def financial_team_supervisor(state: SubTeamState) -> dict:
    """Supervisor for the financial analysis sub-team."""
    decision = market_team_structured_llm.invoke([
        SystemMessage(content="""You supervise a financial analysis team with two analysts:
- revenue_analyst: Evaluates revenue model, unit economics, growth metrics
- risk_analyst: Assesses financial, market, regulatory, execution, and competitive risks

Review the conversation. Route to whichever analyst hasn't reported yet.
If both have reported, route to FINISH.
Always ensure both analysts contribute before finishing."""),
        *state["messages"],
    ])
    
    next_node = decision.next.lower().strip()
    if "finish" in next_node:
        summary = llm.invoke([
            SystemMessage(content="Compile a concise financial analysis summary combining revenue and risk findings. Include key metrics, risk ratings, and investment recommendation. 2-3 paragraphs max."),
            *state["messages"],
        ])
        return {
            "next": "FINISH",
            "team_output": summary.content,
            "messages": [AIMessage(content="[Financial Supervisor] Team complete. Summary compiled.")],
        }
    
    if "revenue" in next_node:
        return {"next": "revenue_analyst", "messages": [AIMessage(content="[Financial Supervisor] Routing to revenue analyst.")]}
    elif "risk" in next_node:
        return {"next": "risk_analyst", "messages": [AIMessage(content="[Financial Supervisor] Routing to risk analyst.")]}
    else:
        return {"next": "revenue_analyst", "messages": [AIMessage(content="[Financial Supervisor] Defaulting to revenue analyst.")]}

def route_financial_team(state: SubTeamState) -> str:
    next_node = state.get("next", "FINISH")
    if next_node == "FINISH":
        return END
    return next_node

# Build Financial Analysis subgraph
financial_graph = StateGraph(SubTeamState)
financial_graph.add_node("supervisor", financial_team_supervisor)
financial_graph.add_node("revenue_analyst", revenue_analyst)
financial_graph.add_node("risk_analyst", risk_analyst)

financial_graph.add_edge(START, "supervisor")
financial_graph.add_conditional_edges("supervisor", route_financial_team, {
    "revenue_analyst": "revenue_analyst",
    "risk_analyst": "risk_analyst",
    END: END,
})
financial_graph.add_edge("revenue_analyst", "supervisor")
financial_graph.add_edge("risk_analyst", "supervisor")

financial_analysis_app = financial_graph.compile()
print("Financial Analysis sub-team compiled!")

## 4. Top-Level Orchestrator

The orchestrator coordinates both sub-teams. The key challenge is **state mapping** — translating between the parent's `OrchestratorState` and each child's `SubTeamState`.

Each sub-team wrapper:
1. Extracts relevant messages from the parent state
2. Invokes the compiled sub-team subgraph  
3. Maps the sub-team's output back into the parent state

In [ ]:
def run_market_research(state: OrchestratorState) -> dict:
    """Wrapper that invokes the market research sub-team."""
    # State mapping: parent → child
    child_input = {
        "messages": state["messages"],  # Pass through the main query
        "next": "",
        "team_output": "",
    }
    
    # Run the sub-team
    child_result = market_research_app.invoke(child_input)
    
    # State mapping: child → parent
    return {
        "market_research_output": child_result.get("team_output", "No output generated"),
        "messages": [AIMessage(content=f"[Market Research Team] Completed analysis.\n\nSummary: {child_result.get('team_output', 'No output')[:500]}")],
    }

def run_financial_analysis(state: OrchestratorState) -> dict:
    """Wrapper that invokes the financial analysis sub-team."""
    child_input = {
        "messages": state["messages"],
        "next": "",
        "team_output": "",
    }
    
    child_result = financial_analysis_app.invoke(child_input)
    
    return {
        "financial_analysis_output": child_result.get("team_output", "No output generated"),
        "messages": [AIMessage(content=f"[Financial Analysis Team] Completed analysis.\n\nSummary: {child_result.get('team_output', 'No output')[:500]}")],
    }

print("Sub-team wrapper functions defined")

In [ ]:
class OrchestratorDecision(BaseModel):
    """Top-level routing decision."""
    next: Literal["market_research", "financial_analysis", "synthesis", "FINISH"] = Field(
        description="Next team or step to execute"
    )
    reasoning: str = Field(description="Why this routing decision")

orchestrator_structured_llm = llm.with_structured_output(OrchestratorDecision)

def orchestrator_node(state: OrchestratorState) -> dict:
    """Top-level orchestrator that coordinates sub-teams."""
    market_done = bool(state.get("market_research_output"))
    financial_done = bool(state.get("financial_analysis_output"))
    
    decision = orchestrator_structured_llm.invoke([
        SystemMessage(content=f"""You are the lead orchestrator for a startup due diligence process.

Available teams:
- market_research: Market sizing, trends, competitive landscape analysis
- financial_analysis: Revenue model, unit economics, risk assessment

Status:
- Market research: {"COMPLETED" if market_done else "NOT STARTED"}
- Financial analysis: {"COMPLETED" if financial_done else "NOT STARTED"}

Route to the next team that hasn't completed their analysis.
If both teams are done, route to "synthesis" for the final report.
Both teams MUST complete before synthesis."""),
        *state["messages"],
    ])
    
    return {
        "next": decision.next,
        "messages": [AIMessage(content=f"[Orchestrator] {decision.reasoning} → Routing to {decision.next}")],
    }

def synthesis_agent(state: OrchestratorState) -> dict:
    """Combines both teams' findings into a final investment report."""
    report = llm.invoke([
        SystemMessage(content="""You are a senior investment analyst writing the final due diligence report.

Combine the market research and financial analysis into a comprehensive investment recommendation.

Structure your report as:
1. Executive Summary (2-3 sentences)
2. Market Opportunity (key findings from market research)
3. Financial Assessment (key findings from financial analysis)
4. Key Risks
5. Investment Recommendation (Invest / Pass / Conditional with specific conditions)

Be specific, cite data points, and give a clear recommendation."""),
        HumanMessage(content=f"""Market Research Findings:
{state.get('market_research_output', 'Not available')}

Financial Analysis Findings:
{state.get('financial_analysis_output', 'Not available')}"""),
        *state["messages"][:1],  # Include original query for context
    ])
    
    return {
        "final_report": report.content,
        "messages": [AIMessage(content=f"[Synthesis] Final report compiled.\n\n{report.content}")],
        "next": "FINISH",
    }

print("Orchestrator and synthesis nodes defined")

In [ ]:
def route_orchestrator(state: OrchestratorState) -> str:
    next_node = state.get("next", "FINISH")
    if next_node == "FINISH":
        return END
    return next_node

# Build the top-level graph
full_graph = StateGraph(OrchestratorState)

full_graph.add_node("orchestrator", orchestrator_node)
full_graph.add_node("market_research", run_market_research)
full_graph.add_node("financial_analysis", run_financial_analysis)
full_graph.add_node("synthesis", synthesis_agent)

full_graph.add_edge(START, "orchestrator")
full_graph.add_conditional_edges("orchestrator", route_orchestrator, {
    "market_research": "market_research",
    "financial_analysis": "financial_analysis",
    "synthesis": "synthesis",
    END: END,
})
full_graph.add_edge("market_research", "orchestrator")
full_graph.add_edge("financial_analysis", "orchestrator")
full_graph.add_edge("synthesis", END)

full_app = full_graph.compile()
print("Full hierarchical graph compiled!")

In [ ]:
from IPython.display import display, Image

try:
    display(Image(full_app.get_graph().draw_mermaid_png()))
except Exception:
    print(full_app.get_graph().draw_mermaid())

## 5. Running the Full Due Diligence

Let's evaluate "Acme AI Corp" as a Series B investment. Watch how the orchestrator dispatches work to sub-teams, each sub-team's supervisor routes between its analysts, and the synthesis agent pulls everything together.

In [ ]:
full_result = full_app.invoke({
    "messages": [HumanMessage(content="""Evaluate Acme AI Corp as a potential Series B investment.

Company profile:
- AI-powered diagnostic tools for hospitals
- Founded 2021, 45 employees
- Current ARR: $8M, growing 120% YoY
- 28 hospital customers
- One FDA-cleared product, two more in pipeline
- Seeking $50M Series B at $250M pre-money valuation
- Enterprise SaaS model with usage-based component

Please conduct full market research and financial due diligence.""")],
    "next": "",
    "market_research_output": "",
    "financial_analysis_output": "",
    "final_report": "",
})

print("=" * 80)
print("FULL DUE DILIGENCE: Acme AI Corp — Series B ($50M at $250M)")
print("=" * 80)

# Print execution trace
for msg in full_result["messages"]:
    if isinstance(msg, HumanMessage):
        print(f"\n{'QUERY':>20}: {msg.content[:150]}...")
    elif isinstance(msg, AIMessage):
        prefix = msg.content.split("]")[0] + "]" if "]" in msg.content else "AGENT"
        content = msg.content[len(prefix):].strip() if "]" in msg.content else msg.content
        print(f"\n{prefix:>30}: {content[:250]}")

In [ ]:
print("=" * 80)
print("STATE INSPECTION")
print("=" * 80)

print("\n📊 MARKET RESEARCH OUTPUT:")
print("─" * 40)
print(full_result.get("market_research_output", "N/A")[:600])

print("\n\n💰 FINANCIAL ANALYSIS OUTPUT:")
print("─" * 40)
print(full_result.get("financial_analysis_output", "N/A")[:600])

print("\n\n📋 FINAL REPORT:")
print("─" * 40)
print(full_result.get("final_report", "N/A")[:1000])

## 6. When to Use Hierarchical vs Flat

| Factor | Flat (Single Supervisor) | Hierarchical (Teams) |
|--------|-------------------------|---------------------|
| Number of agents | 2-5 | 6+ |
| Domain diversity | Similar domains | **Distinct specializations** |
| Context window pressure | Low | **Reduces per-supervisor context** |
| Development complexity | Simple | More complex |
| Debugging | Easy to trace | Requires per-team inspection |
| Scalability | Limited | **Highly scalable** |
| State isolation | Shared everything | **Scoped per team** |
| Reusability | Low | **Sub-teams are independently testable** |

### Key Architectural Principles

1. **State boundaries matter** — Sub-teams should have their own state type. Don't leak internal state to the parent.
2. **Test sub-teams independently** — Each sub-team is a compiled graph that can be invoked and tested on its own.
3. **State mapping is the glue** — The wrapper functions that translate between parent and child state are the critical integration points.
4. **Summaries, not transcripts** — Pass team summaries to the parent, not raw message history. This keeps the orchestrator's context clean.
5. **Two levels is usually enough** — Going deeper than 2 levels of hierarchy adds complexity without proportional benefit.

### Next Steps
- **Collaboration Patterns** → When agents within a team need to debate or vote rather than just take turns
- **Custom State Machines** → When you need explicit lifecycle stages with retry logic
- **Human in the Loop** → When you need human approval at certain decision points